# 上下文管理器和else块

In [1]:
for i in range(8):
    if i == 3:
        continue
    print(i)
else:
    print('print 3 individually')

0
1
2
4
5
6
7
print 3 individually


In [2]:
with open('bisect_test.py') as fp:
    # 对open()函数来说，必须加上as子句，以便获取文件的引用
    src = fp.read(60)

In [3]:
len(src)

60

In [4]:
fp

<_io.TextIOWrapper name='bisect_test.py' mode='r' encoding='UTF-8'>

In [5]:
fp.closed, fp.encoding

(True, 'UTF-8')

In [6]:
fp.read(60)

ValueError: I/O operation on closed file.

## with语句的目的是简化try/finally模式

In [7]:
class LookingGlass:
    
    def __enter__(self):
        # 当with语句开始执行时，解释器会在上下文管理器对象上调用__enter__方法
        import sys
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'JABBERWOCKY'
    
    def reverse_write(self, text):
        self.original_write(text[::-1])
        
    def __exit__(self, exc_type, exc_value, traceback):    
        # 当with语句运行结束后，解释器通常需要调用sys.exc_info()，得到的就是__exit__接收的这三个参数，用来判断做什么清理工作。
        import sys
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero!')
            return True
    

In [8]:
with LookingGlass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [9]:
what

'JABBERWOCKY'

In [10]:
print('Back to normal.')

Back to normal.


In [11]:
manager = LookingGlass()
manager
# 接下来书中的内容请在命令行中进行

## contextlib模块中的实用工具

In [12]:
import contextlib


@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])
        
    sys.stdout.write = reverse_write
    yield 'JABBERWOCKY'
    sys.stdout.write = original_write

In [13]:
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [14]:
what

'JABBERWOCKY'

In [17]:
import contextlib


@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])
        
    sys.stdout.write = reverse_write
    msg = ''
    try:
        yield 'JABBERWOCKY'    # 要把yield语句放在try/finally语句中，这是不可避免的
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero!'    
        # 装饰器提供的__exit__方法假定发给生成器的所有异常都得到处理了，不需要return True来显示处理异常!
        # raise ZeroDivisionError    如果不想让@contextmanager压制异常，必须显示重新抛出异常
    finally:    # 解释器通过生成器的throw方法在yield表达式处抛出异常，默认异常被处理，finally子句会被执行
        sys.stdout.write = original_write
        if msg:
            print(msg)

In [18]:
# 有关于上述的looking_glass上下文管理器的使用示例请在命令行中进行